In [1]:
%env CUDA_VISIBLE_DEVICES=7
%env OMP_NUM_THREADS=8
%env HF_HOME=/mnt/LLM
!git clone https://github.com/Vahe1994/AQLM AQLM
!wget https://raw.githubusercontent.com/galqiwi/linear-layer-compression-pipeline/refs/heads/main/grids/EDEN6-4096.pt -O EDEN6-4096.pt
!wget https://gist.githubusercontent.com/galqiwi/d8fdeb2c6603ad3e54d72a0801416ad3/raw/3eb0762bebe0e95df5ad17b2f3fa9f4ef3a241ac/edenn.py -O edenn.py

env: CUDA_VISIBLE_DEVICES=7
env: OMP_NUM_THREADS=8
env: HF_HOME=/mnt/LLM
fatal: destination path 'AQLM' already exists and is not an empty directory.
--2025-01-31 04:34:09--  https://raw.githubusercontent.com/galqiwi/linear-layer-compression-pipeline/refs/heads/main/grids/EDEN6-4096.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99499 (97K) [application/octet-stream]
Saving to: ‘EDEN6-4096.pt’

EDEN6-4096.pt       100%[===================>]  97.17K  --.-KB/s    in 0.06s   

2025-01-31 04:34:10 (1.48 MB/s) - ‘EDEN6-4096.pt’ saved [99499/99499]

--2025-01-31 04:34:10--  https://gist.githubusercontent.com/galqiwi/d8fdeb2c6603ad3e54d72a0801416ad3/raw/3eb0762bebe0e95df5ad17b2f3fa9f4ef3a241ac/edenn.py
Resolving gist.githubusercontent.com (gist.

In [2]:
import random
import math

import torch
import torch.nn as nn
from fast_hadamard_transform import hadamard_transform
from edenn import higgs_quantize_dequantize, pad_to_block, HadLinear


@torch.no_grad()
def quantize_linear_weight(weight: torch.Tensor, hadamard_groupsize: int, edenn_d: int, edenn_n: int):
    weight = weight.to(dtype=torch.float32, device='cuda' if X.device.type != 'cuda' else X.device)
    # Pad to Hadamard transform size
    weight = pad_to_block(weight, [1], hadamard_groupsize)
    
    # Scale and Hadamard transform
    mult = weight.shape[1] // hadamard_groupsize
    weight = weight.reshape(-1, mult, hadamard_groupsize)
    scales = torch.linalg.norm(weight, axis=-1)
    weight = hadamard_transform(weight) / scales[:, :, None]
    
    # Pad to edenn_d and project
    weight = pad_to_block(weight, [2], edenn_d).reshape(weight.shape[0], mult, -1, edenn_d)

    for i in range(0, weight.shape[0], 64):
        weight[i: i + 64], entropy = higgs_quantize_dequantize(weight[i:i+64], edenn_d, edenn_n)
    weight = weight.reshape(weight.shape[0], mult, -1)
    
    # Cut the padded values
    weight = weight[...,:hadamard_groupsize]
    
    # Unscale
    weight = (weight * scales[:, :, None]).reshape(weight.shape[0], -1)
    
    return HadLinear(weight.half(), hadamard_groupsize), entropy


def quantize_dequantize(weight: torch.Tensor, hadamard_groupsize: int, edenn_d: int, edenn_n: int):
    output_layer, _ = quantize_linear_weight(weight, hadamard_groupsize, edenn_d, edenn_n)
    return output_layer(torch.eye(weight.shape[1], device='cuda').half()).T.detach().contiguous().clone().to(weight.device)


X = torch.randn(64, 1024) * 0.01
- math.log((quantize_dequantize(X, 1024, 6, 4096) - X).norm() / X.norm()) / math.log(2)

/home/jheuristic/exp/kv-caches-quantization/edenn.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  GRIDS[dim][size] = torch.load(file)


1.7271932614287617

In [3]:
import sys
sys.path.insert(0, "AQLM")
import time
from typing import Optional, Dict, Any, Sequence, Tuple

import copy
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn
import sklearn.linear_model
from sklearn.model_selection import train_test_split

import transformers
from tqdm.auto import tqdm, trange
import main as calibration_utils

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


class args:  # wannabe argparse namespace
    model_name = "unsloth/Llama-3.2-3B"
    torch_dtype = "auto"
    compute_dtype = None  # used for computing activations
    dataset = "pajama"
    total_nsamples = 256
    valid_nsamples = 32
    model_seqlen = 8192
    seed = 42
    offload_activations = True  # if True, store hidden states in RAM
    devices = [device]
    wandb = False  # <-- to be implemented
    percdamp = 1e-3
    chunk_size = 4096
    
    hadamard_groupsize = 1024
    edenn_d = 6
    edenn_n = 4096

In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained(args.model_name)
model = transformers.AutoModelForCausalLM.from_pretrained(
    args.model_name, torch_dtype=args.torch_dtype, low_cpu_mem_usage=True, use_cache=False
)
layers = calibration_utils.get_layers(model)

data = calibration_utils.get_loaders(
    args.dataset,
    nsamples=args.total_nsamples,
    seed=args.seed,
    model_path=args.model_name,
    seqlen=args.model_seqlen,
)

Loading red_pajama from togethercomputer/RedPajama-Data-1T-Sample


Using the latest cached version of the module from /mnt/LLM/modules/datasets_modules/datasets/togethercomputer--RedPajama-Data-1T-Sample/6ea3bc8ec2e84ec6d2df1930942e9028ace8c5b9d9143823cf911c50bbd92039 (last modified on Sat Jan 13 21:19:54 2024) since it couldn't be found locally at togethercomputer/RedPajama-Data-1T-Sample, or remotely on the Hugging Face Hub.
Making red_pajama calibration set:   6%|█████████████████▏                                                                                                                                                                                                                                                                                    | 15/256 [00:03<01:09,  3.46it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (216598 > 131072). Running this sequence through the model will result in indexing errors
                                                                                 

Loaded data from pajama; len(data)=256 sequences


In [5]:
def update_outs_inplace_(
    args, layer: nn.Module, inps: Sequence[torch.Tensor], outs: Sequence[torch.Tensor], **kwargs
):
    """
    Forward a single layer on a collection of :inps: to update :outs: in-place
    see docs at https://github.com/Vahe1994/AQLM/blob/main/main.py#L530
    """
    assert len(inps) == len(outs) == len(args.devices)
    if len(args.devices) == 1:
        return calibration_utils.update_outs(layer, inps[0], outs[0], **kwargs)
    else:
        return calibration_utils.update_outs_parallel(args.devices, layer, inps, outs, **kwargs)


class OutputCatcher(nn.Module):
    """Wraps a layer to catch its output tensors from one or more forward passes"""
    def __init__(self, inner):
        super().__init__()
        self.inner = inner
        self.outputs = []
    def forward(self, inp, **kwargs):
        output = self.inner(inp)
        self.outputs.append(output.to('cpu' if args.offload_activations else device, copy=True))
        return output


def get_predictor(predictor_inputs: Sequence[torch.Tensor], targets: Sequence[torch.Tensor]):
    """Train predictor, return the predictor itself and train/valid errors"""
    X = torch.stack(predictor_inputs, dim=0)
    Y = torch.stack(targets, dim=0)
    assert X.shape[:-1] == Y.shape[:-1] == (args.total_nsamples, args.model_seqlen)
    train_ids, valid_ids = torch.randperm(
        len(X), generator=torch.Generator(X.device).manual_seed(args.seed), device=X.device
    ).split_with_sizes((args.total_nsamples - args.valid_nsamples, args.valid_nsamples))
    X_train, X_valid, Y_train, Y_valid = X[train_ids], X[valid_ids], Y[train_ids], Y[valid_ids]
    X_train, X_valid, Y_train, Y_valid = [t.flatten(0, -2) for t in (X_train, X_valid, Y_train, Y_valid)]
    weight, bias = fit_linear_regression(
        X_train, Y_train, reg_rate=args.percdamp, fit_intercept=True,
        compute_device=device, compute_dtype=torch.float32, chunk_size=args.chunk_size
    )
    predictor = nn.Linear(*weight.shape[::-1], dtype=X.dtype, device=device)
    with torch.no_grad():
        predictor.weight[...] = weight
        predictor.bias[...] = bias
    mse_train, ev_train = compute_relative_mse(
        predictor, X_train, Y_train, compute_device=device, chunk_size=args.chunk_size)
    mse_valid, ev_valid = compute_relative_mse(
        predictor, X_valid, Y_valid, compute_device=device, chunk_size=args.chunk_size)
    return predictor, mse_train, mse_valid, ev_train, ev_valid


@torch.no_grad()
def get_dequant_values(predictor_inputs, values, pred: nn.Module):
    """Return a list of reconstructed (quantized-dequantized with predictor) tensors; values can be K or V"""
    assert len(predictor_inputs) == len(values)
    values_dequantized = []
    for i in trange(len(predictor_inputs), desc='get_dequant_values', leave=False):
        predictor_inputs_i = predictor_inputs[i].to(device, non_blocking=True)
        values_i = values[i].to(device, non_blocking=True)
        values_pred_i = pred(predictor_inputs_i)
        values_delta = values_i - values_pred_i
        values_delta_dequant_i = quantize_dequantize(
            values_delta.flatten(0, -2), args.hadamard_groupsize, args.edenn_d, args.edenn_n,
        ).reshape(values_delta.shape)
        values_dequantized.append(
            (values_delta_dequant_i + values_pred_i).to(values[i].device, non_blocking=True))
    return values_dequantized


@torch.no_grad()
def compute_relative_mse(
    predictor: nn.Module, X: torch.Tensor, Y: torch.Tensor, chunk_size: int = None,
    compute_device: Optional[device] = None, compute_dtype: torch.dtype = None
) -> Tuple[float, float]:
    """Compute ||predictor(X) - Y||^2 / ||Y||^2 in a memory-efficient manner"""
    if compute_device is None:
        compute_device = next(predictor.parameters()).device

    if chunk_size is None:
        assert False
    else:
        
        numerator = denominator = variance_numerator = variance_denominator = 0
        for chunk_start in trange(0, len(X), chunk_size, desc='compute_relative_mse', leave=False):
            xb, yb = [tensor[chunk_start: chunk_start + chunk_size].to(
                device=compute_device, dtype=compute_dtype, non_blocking=True)
                      for tensor in (X, Y)]
            deltas = (predictor(xb) - yb)
            numerator += deltas.norm().square().item()
            denominator += yb.norm().square().item()
            
            variance_numerator += deltas.flatten(0, -2).var(dim=0).mean().item() * yb.numel()
            variance_denominator += yb.flatten(0, -2).var(dim=0).mean().item() * yb.numel()
        return numerator / denominator, (1 - variance_numerator / variance_denominator)


@torch.no_grad()
def fit_linear_regression(
        X: torch.Tensor,
        Y: torch.Tensor, *,
        reg_rate: float = 0,
        fit_intercept: bool = True,
        compute_device: Optional[torch.device] = None,
        compute_dtype: torch.dtype = None,
        chunk_size: Optional[int] = None,
        verbose: bool = False,
):
    """
    Multivariate linear regression with exactsolution
    :param X: input, shape: [nsamples, in_features]
    :param Y: target, shape: [nsamples, num_targets]
    :param reg_rate: regularize weights by this value, in proportion to mean square input
    :param fit_intercept: if True (default), learn the bias term as in the regular least-squares
        if False, force bias term to be zeros -- but still return the resulting all-zero vector
    :param compute_device: optionally transfer the covariance matrix to this device and compute inverse there
        Computing pinverse of large matrices on CPU can take hours
    :param compute_dtype: optionally cast tensors to this dtype for computations (memory-efficiently)
    :returns: (W, b) such that Y ~ (X @ W.T + b)
    """
    assert X.ndim == Y.ndim == 2 and X.shape[0] == Y.shape[0], "X, Y must be [nsamples, in/out_features]"
    (nsamples, in_features), (_, out_features), source_device = X.shape, Y.shape, Y.device
    orig_dtype = X.dtype

    if chunk_size is None:
        X = X.to(compute_dtype)
        if fit_intercept:
            X = torch.cat([X, torch.ones(X.shape[0], 1, device=X.device)], dim=1)
        CXX = (X.T @ X).to(compute_device)  # [in_features + 1, in_features + 1], aka MSE hessian
        # add column of ones
    else:
        CXX = torch.zeros(in_features + 1, in_features + 1, device=compute_device, dtype=compute_dtype or X.dtype)
        for chunk_start in trange(0, nsamples, chunk_size, desc='fit_linear_regression::CXX',
                                  leave=False, disable=not verbose):
            xb = X[chunk_start: chunk_start + chunk_size].to(
                device=compute_device, dtype=compute_dtype, non_blocking=True)
            if fit_intercept:
                xb = torch.cat([xb, torch.ones(xb.shape[0], 1, device=xb.device)], dim=1)
            CXX = torch.addmm(CXX, xb.T, xb, out=CXX)
            del xb

    if reg_rate > 0:
        ix = torch.arange(len(CXX), device=compute_device or source_device)
        CXX[ix, ix] += reg_rate * abs(torch.diag(CXX)).mean()
        del ix

    CXX_pinv = torch.pinverse(CXX)
    del CXX

    if chunk_size is None:
        CXY = (X.T @ Y).to(compute_device)  # [in_features, out_features]
        del X, Y
    else:
        CXY = torch.zeros(in_features + 1, out_features, device=compute_device, dtype=compute_dtype or X.dtype)
        for chunk_start in trange(0, nsamples, chunk_size, desc='fit_linear_regression::CXY',
                                  leave=False, disable=not verbose):
            xb, yb = [tensor[chunk_start: chunk_start + chunk_size].to(
                device=compute_device, dtype=compute_dtype, non_blocking=True)
                for tensor in (X, Y)]
            if fit_intercept:
                xb = torch.cat([xb, torch.ones(xb.shape[0], 1, device=xb.device)], dim=1)
            CXY = torch.addmm(CXY, xb.T, yb, out=CXY)
            del xb, yb
        del X, Y

    W = (CXX_pinv @ CXY).T.to(source_device, dtype=orig_dtype)

    if fit_intercept:
        W, bias = W[:, :-1], W[:, -1]
    else:
        bias = None

    return W, bias


In [6]:
from collections import defaultdict
RESULTS = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))
#         layer_index:        target_name:        input_name:  (train_relmse={}, valid_relmse={})


target_factories = dict(
    keys = lambda: attn_keys,
    values = lambda: attn_values,
)
input_factories = dict(
    const = lambda: [torch.zeros_like(k) for k in attn_keys],
    keys = lambda: attn_keys,
    prev_keys = lambda: old_attn_keys,
    preprev_keys = lambda: preprev_attn_keys,
    prepreprev_keys = lambda: prepreprev_attn_keys,
    
    values = lambda: attn_values,
    prev_values = lambda: old_attn_values,
    preprev_values = lambda: preprev_attn_values,
    prepreprev_values = lambda: prepreprev_attn_values,
    
    prev_keys_and_values = lambda: [torch.cat([ki, vi], dim=-1) for ki, vi in zip(
        old_attn_keys, old_attn_values)],
    past_3_keys = lambda: [torch.cat([ki, kj, kk], dim=-1) for ki, kj, kk in zip(
        old_attn_keys, prepreprev_attn_keys, prepreprev_attn_keys)],
    past_3_values = lambda: [torch.cat([ki, kj, kk], dim=-1) for ki, kj, kk in zip(
        old_attn_values, prepreprev_attn_values, prepreprev_attn_values)],
    
    prev_token_keys = lambda: [torch.cat((torch.zeros_like(k_i[:1, :]), k_i[:-1, :]), dim=0)
                               for k_i in old_attn_keys],
    prev_token_values = lambda: [torch.cat((torch.zeros_like(k_i[:1, :]), k_i[:-1, :]), dim=0)
                                 for k_i in old_attn_values],
    preprev_token_keys = lambda: [torch.cat((torch.zeros_like(k_i[:2, :]), k_i[:-2, :]), dim=0)
                               for k_i in old_attn_keys],
    preprev_token_values = lambda: [torch.cat((torch.zeros_like(k_i[:2, :]), k_i[:-2, :]), dim=0)
                                 for k_i in old_attn_values],
    prepreprev_token_keys = lambda: [torch.cat((torch.zeros_like(k_i[:3, :]), k_i[:-3, :]), dim=0)
                               for k_i in old_attn_keys],
    prepreprev_token_values = lambda: [torch.cat((torch.zeros_like(k_i[:3, :]), k_i[:-3, :]), dim=0)
                                 for k_i in old_attn_values],
)

In [ ]:
inps, forward_args = calibration_utils.get_inps(
    model, data, args.model_seqlen, args.devices, args.offload_activations)

for k, v in forward_args.items():
    forward_args[k] = v.to(args.devices[0]) if isinstance(v, torch.Tensor) else v

outs = [torch.zeros_like(inp_tensor, pin_memory=inp_tensor.is_pinned()) for inp_tensor in inps]
old_attn_keys = None
preprev_attn_keys = None
prepreprev_attn_keys = None
old_attn_values = None
preprev_attn_values = None
prepreprev_attn_values = None

predictors_keys = {}
predictors_values = {}

for layer_index in range(len(layers)):
    print(f"\n---------------- Layer {layer_index} of {len(layers)} ----------------")
    start_time = time.perf_counter()

    layer_device_original = next(layers[layer_index].parameters()).device
    layer_dtype_original = next(layers[layer_index].parameters()).dtype
    layer = layers[layer_index].to(device=args.devices[0], dtype=args.compute_dtype or layer_dtype_original)
    
    layer.self_attn.k_proj = OutputCatcher(layer.self_attn.k_proj)
    layer.self_attn.v_proj = OutputCatcher(layer.self_attn.v_proj)
    
    update_outs_inplace_(args, layer, inps, outs, **forward_args, compute_mse=False)

    attn_keys = layer.self_attn.k_proj.outputs
    assert all(elem.shape[0] == 1 for elem in attn_keys)
    attn_keys = [elem[0] for elem in attn_keys]
    
    attn_values = layer.self_attn.v_proj.outputs
    assert all(elem.shape[0] == 1 for elem in attn_values)
    attn_values = [elem[0] for elem in attn_values]
    
    layer.self_attn.k_proj = layer.self_attn.k_proj.inner
    layer.self_attn.v_proj = layer.self_attn.v_proj.inner
    
    layers[layer_index] = layer.to(device=layer_device_original, dtype=layer_dtype_original)
    del layer
    torch.cuda.empty_cache()
    
    inps, outs = outs, inps

    if layer_index == 0:
        old_attn_keys = attn_keys
        old_attn_values = attn_values
        preprev_attn_keys = prepreprev_attn_keys = prepreprev_attn_values = preprev_attn_values = [
            torch.zeros_like(x) for x in attn_keys]
        continue

    ### training predictor below ###
    for target_name, make_targets in target_factories.items():
        for input_name, make_inputs in input_factories.items():
            _predictor, mse_train, mse_valid, ev_train, ev_valid = get_predictor(make_inputs(), make_targets())
            print(f"{input_name} -> {target_name} ")
            print(f'\t| train relMSE: {mse_train:.8f} \t| valid relMSE: {mse_valid:.8f}')
            print(f'\t| train EVR: {ev_train:.8f} \t| valid EVR: {ev_valid:.8f}')
            RESULTS[layer_index][target_name][input_name]['train_relative_mse'] = mse_train
            RESULTS[layer_index][target_name][input_name]['valid_relative_mse'] = mse_valid
            RESULTS[layer_index][target_name][input_name]['train_explained_variance_ratio'] = ev_train
            RESULTS[layer_index][target_name][input_name]['valid_explained_variance_ratio'] = ev_valid
            del _predictor
    
    prepreprev_attn_keys, prepreprev_attn_values = preprev_attn_keys, preprev_attn_values
    preprev_attn_keys, preprev_attn_values = old_attn_keys, old_attn_values
    old_attn_keys, old_attn_values = attn_keys, attn_values

catching layer inputs from data

---------------- Layer 0 of 28 ----------------


calc outs after quantization:   0%|          | 0/256 [00:00<?, ?it/s]

The attention layers in this model are transitioning from computing the RoPE embeddings internally through `position_ids` (2D tensor with the indexes of the tokens), to using externally computed `position_embeddings` (Tuple of tensors, containing cos and sin). In v4.46 `position_ids` will be removed and `position_embeddings` will be mandatory.



---------------- Layer 1 of 28 ----------------


calc outs after quantization:   0%|          | 0/256 [00:00<?, ?it/s]

...

compute_relative_mse:   0%|          | 0/64 [00:00<?, ?it/s]

prev_token_keys -> values 
	| train relMSE: 0.69317927 	| valid relMSE: 0.71090500
	| train EVR: 0.20713987 	| valid EVR: 0.19859346


compute_relative_mse:   0%|          | 0/448 [00:00<?, ?it/s]

compute_relative_mse:   0%|          | 0/64 [00:00<?, ?it/s]

prev_token_values -> values 
	| train relMSE: 0.70402898 	| valid relMSE: 0.72304666
	| train EVR: 0.19584977 	| valid EVR: 0.18697981


In [ ]:
import json
with open("results.json", 'w') as f:
    json.dump(RESULTS, f)